In [1]:
import pandas as pd
import numpy as np
import json, glob
import itertools
import os
from pathlib import Path

In [40]:
exp_dir = Path('../data/experiments/12-03-2021/13_02_52')
manifest = exp_dir / 'test.json'
res_file = exp_dir / 'all_results.json'

In [41]:
results = json.load(open(res_file))
prompts = json.load(open(manifest))
assert len(results) == len(prompts)
len(prompts)

20

In [42]:
a = list(itertools.product(['-2=>', '-1=>', 'False=>', 'True=>'], ['-2', '-1', 'False', 'True']))
a = [''.join(b) for b in a]

In [43]:
def split_prob(prob, keep_filename=True):
    tmp = prob.split('Prompt-Summarization/data/')[1]
    if keep_filename:
        return tmp
    return os.path.split(tmp)[0]

In [44]:
probs = {split_prob(prompt, False) for prompt in prompts}
probs

{'competition/2053',
 'competition/2073',
 'competition/2187',
 'competition/2197',
 'competition/2360',
 'interview/1312',
 'interview/1401',
 'interview/1411',
 'interview/1462',
 'interview/1922'}

In [45]:
questions = [split_prob(prompt) for prompt in prompts]
questions

['competition/2073/expert.txt',
 'competition/2073/question.txt',
 'interview/1401/expert.txt',
 'interview/1401/question.txt',
 'interview/1462/expert.txt',
 'interview/1462/question.txt',
 'interview/1312/expert.txt',
 'interview/1312/question.txt',
 'interview/1411/expert.txt',
 'interview/1411/question.txt',
 'competition/2360/expert.txt',
 'competition/2360/question.txt',
 'competition/2053/expert.txt',
 'competition/2053/question.txt',
 'competition/2197/expert.txt',
 'competition/2197/question.txt',
 'competition/2187/expert.txt',
 'competition/2187/question.txt',
 'interview/1922/expert.txt',
 'interview/1922/question.txt']

In [46]:
acc = pd.DataFrame(0, index=questions, columns=[-2, -1, False, True])
for res, prompt in zip(results.values(), questions):
    i = 0
    for r in res[0]:
        i += 1
        acc.loc[prompt, r] +=1

In [47]:
acc

,-2,-1,False,True
competition/2073/expert.txt,0,0,12,2
competition/2073/question.txt,0,0,4,10
interview/1401/expert.txt,0,0,0,1
interview/1401/question.txt,0,0,0,1
interview/1462/expert.txt,0,0,0,1
interview/1462/question.txt,0,0,0,1
interview/1312/expert.txt,0,0,1,0
interview/1312/question.txt,0,0,1,0
interview/1411/expert.txt,0,0,1,0
interview/1411/question.txt,1,0,0,0


In [48]:
acc_df = acc.copy(deep=False)
acc_df['acc'] = 100*acc_df[True]/acc_df.sum(axis=1)
acc_df

,-2,-1,False,True,acc
competition/2073/expert.txt,0,0,12,2,14.285714
competition/2073/question.txt,0,0,4,10,71.428571
interview/1401/expert.txt,0,0,0,1,100.000000
interview/1401/question.txt,0,0,0,1,100.000000
interview/1462/expert.txt,0,0,0,1,100.000000
interview/1462/question.txt,0,0,0,1,100.000000
interview/1312/expert.txt,0,0,1,0,0.000000
interview/1312/question.txt,0,0,1,0,0.000000
interview/1411/expert.txt,0,0,1,0,0.000000
interview/1411/question.txt,1,0,0,0,0.000000


In [49]:
strict_accuracy = 100*sum(acc_df['acc'] >= 100)/len(acc_df)
strict_accuracy

20.0

In [50]:
acc_df = acc.copy(deep=False)
correct = acc_df[True].sum()
total   = acc_df.sum().sum()
tmp = 100 * correct/total
acc_df['acc'] = 100 * acc_df[True]/acc_df.sum(axis=1)
strict_acc = 100 * sum(acc_df['acc'] >= 100)/len(acc_df)

In [51]:
tmp, strict_acc, correct, total

(23.958333333333332, 20.0, 23, 96)

In [30]:
acc_df['acc']

studio21_generated/competition/2002/expert.txt            0.000000
studio21_generated/competition/2002/question.txt          0.000000
studio21_generated/test/competition/3000/expert.txt      16.666667
studio21_generated/test/competition/3000/question.txt     0.000000
competition/2000/expert.txt                              84.210526
competition/2000/question.txt                             0.000000
Name: acc, dtype: float64

In [55]:
comp = acc.groupby(detect_difficulty).sum()

In [56]:
comp

,-2,-1,False,True
competition-expert,0,0,39,5
competition-question,0,0,30,14
interview-expert,0,0,2,2
interview-question,1,0,1,2


In [73]:
comp.sum().sum()

239.1818181818182

In [75]:
acc_df

,-2,-1,False,True,acc
competition/2073/expert.txt,0,0,12,2,14.285714
competition/2073/question.txt,0,0,4,10,71.428571
interview/1401/expert.txt,0,0,0,1,100.000000
interview/1401/question.txt,0,0,0,1,100.000000
interview/1462/expert.txt,0,0,0,1,100.000000
interview/1462/question.txt,0,0,0,1,100.000000
interview/1312/expert.txt,0,0,1,0,0.000000
interview/1312/question.txt,0,0,1,0,0.000000
interview/1411/expert.txt,0,0,1,0,0.000000
interview/1411/question.txt,1,0,0,0,0.000000


In [84]:
tmp = acc[True]
tmp.groupby(detect_difficulty).mean()

competition-expert      1.0
competition-question    2.8
interview-expert        0.4
interview-question      0.4
Name: True, dtype: float64

In [85]:
ser = acc_df['acc'] >= 100

In [86]:
100 * ser.groupby(detect_difficulty).mean()

competition-expert       0.0
competition-question     0.0
interview-expert        40.0
interview-question      40.0
Name: acc, dtype: float64

In [65]:
comp['acc'] = 100 * comp[True]/comp.sum(axis=1)
comp

,-2,-1,False,True,acc
competition-expert,0,0,39,5,11.363636
competition-question,0,0,30,14,31.818182
interview-expert,0,0,2,2,50.000000
interview-question,1,0,1,2,50.000000


In [68]:
100 * (comp['acc'] >= 100)

competition-expert      0
competition-question    0
interview-expert        0
interview-question      0
Name: acc, dtype: int64

In [ ]:
correct = comp[True].sum()
total   = acc_df.sum().sum()
tmp = 100 * correct/total
acc_df['acc'] = 100 * acc_df[True]/acc_df.sum(axis=1)
strict_acc = 100 * sum(acc_df['acc'] >= 100)/len(acc_df)

In [54]:
def detect_difficulty(prompt):
    difficulty = ['competition', 'interview', 'introductory']
    summary = os.path.split(prompt)[-1]
    summary = os.path.splitext(summary)[0]
    for d in difficulty:
        if d in prompt:
            return f'{d}-{summary}'

In [13]:
diff_df = acc.groupby(detect_type).sum()
diff_df

,-2,-1,False,True
competition,1,45,7,17


In [14]:
diff_df['acc'] = 100*diff_df[True]/diff_df.sum(axis=1)
diff_df

,-2,-1,False,True,acc
competition,1,45,7,17,24.285714


In [15]:
def detect_test(prompt):
    t = 'test'
    if t in prompt:
        return t
    return 'train'

In [16]:
test_df = acc.groupby(detect_test).sum()
test_df

,-2,-1,False,True
test,0,7,4,1
train,1,38,3,16


In [52]:
better = pd.DataFrame(0, index=probs, columns=a)
worse = pd.DataFrame(0, index=probs, columns=a)
same = pd.DataFrame(0, index=probs, columns=a)
for k, v in itertools.filterfalse(lambda x: int(x[0]) % 2, results.items()):
    k = int(k)
    if questions[k].endswith('expert.txt'):
        summary = v[0]
        original = results[str(k+1)][0]
    else:
        summary = results[str(k+1)][0]
        original = v[0]
    # The len is different if one had a -2 error.
    if len(original) < len(summary):
        original *= len(summary)
    if len(original) > len(summary):
        summary *= len(original)
    assert len(original) == len(summary), f'len must be the same. orig = {len(original)}\
    summ = {len(summary)}'
    num = os.path.split(questions[k])[0]
    #prob_idx[num] = k
    for orig, summ in zip(original, summary):
        col = f'{orig}=>{summ}'
        if summ > orig:
            better.loc[num, col] += 1
        elif summ < orig:
            worse.loc[num, col] += 1
        else:
            same.loc[num, col] += 1

In [53]:
better = better.loc[:, ~(better==0).all()]
worse = worse.loc[:, ~(worse==0).all()]
same = same.loc[:, ~(same==0).all()]

In [54]:
same

,-1=>-1
competition/2000,0
studio21_generated/test/competition/3000,1
studio21_generated/competition/2002,0


In [55]:
worse

,False=>-1
competition/2000,0
studio21_generated/test/competition/3000,4
studio21_generated/competition/2002,0


In [56]:
better

,-2=>-1,-1=>False,-1=>True
competition/2000,0,3,16
studio21_generated/test/competition/3000,0,0,1
studio21_generated/competition/2002,19,0,0
